# Notebook 13: Regeneration Loop (Self-Correcting Agent)

Purpose:
- Enable NEEL to safely regenerate responses when Reflection flags issues
- Preserve usefulness without compromising safety
- Complete NEEL’s closed-loop reasoning system

This notebook finalizes NEEL’s backend intelligence pipeline.

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
import os
from dotenv import load_dotenv
from google import genai

In [2]:
load_dotenv()

GEMINI_API_KEY = os.getenv("Google_Gemini_Api_Key")
if not GEMINI_API_KEY:
    raise ValueError("Gemini API key not found")

client = genai.Client(api_key=GEMINI_API_KEY)

In [3]:
class AgentState(TypedDict):
    profile: dict
    memory: dict
    analytics: dict
    supervisor: dict
    llm_response: str
    reflection: dict

In [4]:
user_profile = {
    "user_id": "user_001",
    "name": "Karan",
    "education": "MCA",
    "goal": "Become an ML Engineer",
    "priority": "Learning + Health"
}

In [5]:
analytics_summary = {
    "avg_work_minutes": 421,
    "avg_leisure_minutes": 255,
    "avg_sleep_minutes": 426,
    "avg_exercise_minutes": 64,
    "mean_productivity": 82
}

In [6]:
supervisor_decision = {
    "status": "ALLOW",
    "confidence": "LOW",
    "warnings": [
        "Productivity instability detected",
        "Conflict: high output but reported fatigue"
    ]
}

In [7]:
memory_payload = {
    "short_term": {
        "days_observed": 5,
        "avg_work_minutes": 490,
        "avg_sleep_minutes": 356,
        "avg_exercise_minutes": 18,
        "mean_productivity": 81,
        "productivity_trend": "stable"
    },
    "reflective": "Sleep duration has been consistently low; Physical activity levels are minimal",
    "profile": user_profile
}

In [8]:
def build_reasoning_prompt(profile, analytics, supervisor):
    warnings_text = (
        "\n".join(f"- {w}" for w in supervisor["warnings"])
        if supervisor["warnings"]
        else "None"
    )

    return f"""
You are NEEL, a cautious and responsible AI assistant.

USER CONTEXT:
- Name: {profile['name']}
- Education: {profile['education']}
- Goal: {profile['goal']}
- Priority: {profile['priority']}

ANALYTICS SUMMARY:
- Average Work Time: {analytics['avg_work_minutes']} minutes/day
- Average Leisure Time: {analytics['avg_leisure_minutes']} minutes/day
- Average Sleep Time: {analytics['avg_sleep_minutes']} minutes/day
- Average Exercise Time: {analytics['avg_exercise_minutes']} minutes/day
- Mean Productivity Score: {analytics['mean_productivity']}

SUPERVISOR STATUS:
- Confidence Level: {supervisor['confidence']}
- Warnings:
{warnings_text}

INSTRUCTIONS:
- Do NOT give medical or absolute advice
- Be cautious if confidence is LOW
- Explicitly acknowledge uncertainty
- Do NOT mention ML models or scores
- Keep tone professional and calm

RESPONSE FORMAT:
OBSERVATION:
REASONING:
SUGGESTION:
CONFIDENCE NOTE:
"""

In [9]:
def reflection_agent(llm_text, confidence, user_priority):
    issues = []
    text = llm_text.lower()

    if confidence == "LOW" and "should" in text:
        issues.append("Overconfident language for LOW confidence")

    if "increase" in text and "health" in user_priority.lower():
        issues.append("Suggestion may conflict with health priority")

    if not issues:
        return {"decision": "PASS", "issues": []}

    if len(issues) <= 2:
        return {"decision": "SOFTEN", "issues": issues}

    return {"decision": "REJECT", "issues": issues}

In [10]:
def build_regeneration_prompt(original_response, issues, confidence):
    issues_text = "\n".join(f"- {i}" for i in issues)

    return f"""
The previous response requires revision.

ORIGINAL RESPONSE:
{original_response}

ISSUES IDENTIFIED:
{issues_text}

REWRITE INSTRUCTIONS:
- Be more cautious and tentative
- Avoid prescriptive language
- Explicitly acknowledge uncertainty
- Stay aligned with user priorities
- Do NOT introduce new advice

RESPONSE FORMAT:
OBSERVATION:
REASONING:
SUGGESTION:
CONFIDENCE NOTE:
"""

In [11]:
def analytics_node(state: AgentState) -> AgentState:
    return state


def supervisor_node(state: AgentState) -> AgentState:
    if state["supervisor"]["status"] != "ALLOW":
        state["llm_response"] = "Reasoning blocked by Supervisor."
    return state


def llm_reasoning_node(state: AgentState) -> AgentState:
    prompt = build_reasoning_prompt(
        state["profile"],
        state["analytics"],
        state["supervisor"]
    )

    response = client.models.generate_content(
        model="models/gemini-2.5-flash",
        contents=prompt
    )

    state["llm_response"] = response.text
    return state


def reflection_node(state: AgentState) -> AgentState:
    state["reflection"] = reflection_agent(
        state["llm_response"],
        state["supervisor"]["confidence"],
        state["profile"]["priority"]
    )
    return state


def regeneration_node(state: AgentState) -> AgentState:
    regen_prompt = build_regeneration_prompt(
        state["llm_response"],
        state["reflection"]["issues"],
        state["supervisor"]["confidence"]
    )

    response = client.models.generate_content(
        model="models/gemini-2.5-flash",
        contents=regen_prompt
    )

    state["llm_response"] = response.text
    return state

In [12]:
def route_after_supervisor(state: AgentState):
    return "llm_reasoning" if state["supervisor"]["status"] == "ALLOW" else END


def route_after_reflection(state: AgentState):
    decision = state["reflection"]["decision"]
    if decision == "PASS":
        return END
    if decision == "SOFTEN":
        return "regeneration"
    return END

In [13]:
graph = StateGraph(AgentState)

graph.add_node("analytics", analytics_node)
graph.add_node("supervisor", supervisor_node)
graph.add_node("llm_reasoning", llm_reasoning_node)
graph.add_node("reflection", reflection_node)
graph.add_node("regeneration", regeneration_node)

graph.set_entry_point("analytics")

graph.add_edge("analytics", "supervisor")
graph.add_conditional_edges("supervisor", route_after_supervisor, {
    "llm_reasoning": "llm_reasoning",
    END: END
})

graph.add_edge("llm_reasoning", "reflection")
graph.add_conditional_edges("reflection", route_after_reflection, {
    "regeneration": "regeneration",
    END: END
})

graph.add_edge("regeneration", END)

neel_agent = graph.compile()

In [14]:
initial_state = {
    "profile": user_profile,
    "memory": memory_payload,
    "analytics": analytics_summary,
    "supervisor": supervisor_decision,
    "llm_response": "",
    "reflection": {}
}

final_state = neel_agent.invoke(initial_state)

print("FINAL RESPONSE:\n")
print(final_state["llm_response"])

print("\nREFLECTION RESULT:")
print(final_state["reflection"])

FINAL RESPONSE:

OBSERVATION:
Karan dedicates a substantial portion of his daily time to work activities, alongside regular exercise and a consistent duration of sleep. Despite this considerable output, there is a reported feeling of fatigue, which seems to conflict with his overall efforts. This is further accompanied by indications of instability in his productivity.

REASONING:
Karan's goal to become an ML Engineer and his stated priorities of Learning and Health are central here. While sustained effort is often beneficial for learning, persistent fatigue can hinder cognitive function, learning effectiveness, and overall well-being. The conflict between high output and reported fatigue suggests that the current pattern, while demanding, might be challenging to sustain efficiently or might not be fully supporting his recovery. This could potentially lead to the observed productivity instability, as the quality of work or rest may be compromised over time.

SUGGESTION:
Given the repor

NEEL now:
- Thinks
- Reviews
- Corrects itself
- Then responds

This completes NEEL’s core backend intelligence loop.